<h1><center>Hyperparameter Tuning - 1 </center></h1>

### Test the datasets using pretrained models

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from imblearn.pipeline import Pipeline
from scipy.stats import randint, uniform
from imblearn.over_sampling import SMOTE
import numpy as np
import joblib


In [2]:
# Load datasets
train_dataset = pd.read_csv("train_dataset.csv")
test_dataset = pd.read_csv("test_dataset.csv")

In [3]:
#Drop Patient IDs
train_dataset.drop(["PatientID"], axis=1, inplace=True)
test_dataset.drop(["PatientID"], axis=1, inplace=True)

In [4]:
# Split datasets into features and target
X_train = train_dataset.drop("Severity", axis=1)
y_train = train_dataset["Severity"]

X_test = test_dataset.drop("Severity", axis=1)
y_test = test_dataset["Severity"]

In [5]:

# Perform hyperparameter tuning using RandomizedSearchCV with class_weight='balanced'
def tune_hyperparameters(X_train, y_train):
    model = XGBClassifier(
        use_label_encoder=False,
        eval_metric="mlogloss",
        random_state=42,
    )
    resampler = SMOTE(random_state=42)  
    pipeline = Pipeline(
        [
            ("resampler", resampler),
            ("classifier", model),
      ]
    )

    search = RandomizedSearchCV(
        pipeline,
        param_distributions={
            "classifier__n_estimators": randint(
                50, 600
            ), 
            "classifier__learning_rate": uniform(
                0.005, 0.3
            ),  
            "classifier__max_depth": randint(2, 20),  
            "classifier__min_child_weight": randint(
                1, 15
            ),  
            "classifier__gamma": uniform(0, 2), 
            "classifier__subsample": uniform(0.4, 0.6), 
            "classifier__colsample_bytree": uniform(
                0.4, 0.6
            ),  
            "classifier__reg_alpha": uniform(0, 5),  
            "classifier__reg_lambda": uniform(0, 5),  
            "classifier__tree_method": ["hist"],
        },
        n_iter=150,  
        scoring="accuracy",
        cv=5,
        verbose=1,
        random_state=42,
        n_jobs=-1,
    )
    print("Starting hyperparameter tuning with RandomizedSearchCV...")
    search.fit(X_train, y_train)
    print("Best parameters found from RandomizedSearchCV:", search.best_params_)
    return search.best_estimator_, search.best_params_



In [6]:
# Tune hyperparameters with RandomizedSearchCV
best_model, best_params = tune_hyperparameters(X_train, y_train)


Starting hyperparameter tuning with RandomizedSearchCV...
Fitting 5 folds for each of 150 candidates, totalling 750 fits


/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:21:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:21:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:21:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:21:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:21:22] WARNING

Best parameters found from RandomizedSearchCV: {'classifier__colsample_bytree': 0.988711706816454, 'classifier__gamma': 1.2636270540333367, 'classifier__learning_rate': 0.08294107431924794, 'classifier__max_depth': 14, 'classifier__min_child_weight': 3, 'classifier__n_estimators': 375, 'classifier__reg_alpha': 1.416714741327908, 'classifier__reg_lambda': 0.09213859369108734, 'classifier__subsample': 0.6593444703355074, 'classifier__tree_method': 'hist'}


In [7]:

# Define a refined grid based on the best parameters from RandomizedSearchCV
refined_param_grid = {
    "classifier__n_estimators": [
        max(50, best_params["classifier__n_estimators"] - 50),
        best_params["classifier__n_estimators"],
        best_params["classifier__n_estimators"] + 50,
    ],
    "classifier__learning_rate": [
        max(0.01, best_params["classifier__learning_rate"] * 0.8),
        best_params["classifier__learning_rate"],
        best_params["classifier__learning_rate"] * 1.2,
    ],
    "classifier__max_depth": [
        max(3, best_params["classifier__max_depth"] - 2),
        best_params["classifier__max_depth"],
        best_params["classifier__max_depth"] + 2,
    ],
}


In [8]:
# Perform fine-tuning using GridSearchCV with class_weight='balanced'
def fine_tune_hyperparameters(X_train, y_train, best_model):
    search = GridSearchCV(
        best_model,
        param_grid=refined_param_grid,
        scoring="accuracy",
        cv=5,
        verbose=1,
        n_jobs=-1,
    )
    print("Starting fine-tuning with GridSearchCV...")
    search.fit(X_train, y_train)
    print("Best parameters after fine-tuning:", search.best_params_)
    return search.best_estimator_


In [9]:
# Fine-tune the best model with GridSearchCV
best_model = fine_tune_hyperparameters(X_train, y_train, best_model)


Starting fine-tuning with GridSearchCV...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:03] WARNING

/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/wathsalya/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [22:24:06] WARNING

Best parameters after fine-tuning: {'classifier__learning_rate': 0.06635285945539836, 'classifier__max_depth': 12, 'classifier__n_estimators': 325}


In [10]:
# Evaluate the best model on the test set
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    pred_probabilities = model.predict_proba(X_test)

    # Convert to numpy arrays to ensure consistent handling
    true_labels = np.array(y_test)
    pred_labels = np.array(predictions)

    print("\nEvaluation Results:")
    print(f"Accuracy: {accuracy_score(true_labels, pred_labels):.2f}%")
    print(f"Precision: {precision_score(true_labels, pred_labels, average='weighted'):.2f}%")
    print(f"Recall: {recall_score(true_labels, pred_labels, average='weighted'):.2f}%")
    print(f"F1-Score: {f1_score(true_labels, pred_labels, average='weighted'):.2f}%")
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels))
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_labels, pred_labels))

    # Probability analysis - with fixes
    print("\nProbability Distribution Analysis:")
    
    # Calculate average prediction probability for correct predictions
    correct_indices = pred_labels == true_labels
    if np.any(correct_indices):
        # Fixed: properly extract probabilities for the correct class
        correct_probs = [prob[label] for prob, label in zip(pred_probabilities[correct_indices], true_labels[correct_indices])]
        print(f"Average probability for correct predictions: {np.mean(correct_probs):.4f}")
    
    # Calculate average highest probability for incorrect predictions
    incorrect_indices = ~correct_indices
    if np.any(incorrect_indices):
        incorrect_max_probs = [np.max(prob) for prob in pred_probabilities[incorrect_indices]]
        print(f"Average highest probability for incorrect predictions: {np.mean(incorrect_max_probs):.4f}")
    
    # Find uncertain predictions (highest probability < threshold)
    threshold = 0.7
    uncertain_predictions = [i for i, probs in enumerate(pred_probabilities) 
                            if np.max(probs) < threshold]
    print(f"Number of uncertain predictions (max prob < {threshold}): {len(uncertain_predictions)}")
    
    # Save probability data to CSV for further analysis
    results_df = pd.DataFrame(pred_probabilities, columns=[f'prob_class_{i}' for i in range(pred_probabilities.shape[1])])
    results_df['true_class'] = true_labels
    results_df['predicted_class'] = pred_labels
    results_df['correct'] = correct_indices
    results_df['max_probability'] = [np.max(prob) for prob in pred_probabilities]
    results_df.to_csv('prediction_probabilities_clinical.csv', index=False)
    print("Detailed probability data saved to 'prediction_probabilities_clinical.csv'")
    

In [11]:
evaluate_model(best_model, X_test, y_test)


Evaluation Results:
Accuracy: 0.93%
Precision: 0.93%
Recall: 0.93%
F1-Score: 0.93%

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       254
           1       0.91      0.88      0.89        67
           2       0.89      0.93      0.91        45
           3       0.95      0.91      0.93        64

    accuracy                           0.93       430
   macro avg       0.92      0.92      0.92       430
weighted avg       0.93      0.93      0.93       430


Confusion Matrix:
[[240   6   5   3]
 [  8  59   0   0]
 [  3   0  42   0]
 [  6   0   0  58]]

Probability Distribution Analysis:
Average probability for correct predictions: 0.9114
Average highest probability for incorrect predictions: 0.7947
Number of uncertain predictions (max prob < 0.7): 28
Detailed probability data saved to 'prediction_probabilities_clinical.csv'


In [12]:
# Save the entire pipeline (includes SMOTE + XGBoost)
joblib.dump(best_model, 'best_model_clinical.joblib')
print("Complete pipeline saved to 'best_model_clinical.joblib'")

Complete pipeline saved to 'best_model_clinical.joblib'
